# Classification
Classify malware samples according to their labels using their reduced representations.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 13:42:24.336636: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 13:42:24.872691: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 13:42:25.556077: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:42:25.592223: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:42:25.592272: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The `1-Data-Exploration.ipynb` file should have been run before this, and, as a result, the `processed-data.csv` file should have been generated.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/processed-data.csv")
raw_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,10.802482,0.0,0.0,0.000000,0.000000,12.707260,9.048082,8.721670,...,0.0,13.428146,0.000000,0.0,0.000000,0.000000,0.801215,10.506460,0.024712,0.138089
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,16.677288,0.0,0.0,0.000000,13.798045,0.000000,3.046454,0.000000,...,0.0,11.214973,0.000000,0.0,0.000000,8.833573,6.818852,0.000000,0.000000,0.000000
2,HIVE,122e397dc3a55143bd276d6ff3bc04a05601fbf390aa52...,0.000000,0.0,0.0,6.604103,2.193231,4.128004,2.006744,6.526801,...,0.0,2.912180,10.735530,0.0,0.000000,0.774567,5.789243,0.877978,0.000000,0.000000
3,AGENTB,b390756adcaa12f5abf6c97174d376027a85efd7606ee5...,0.000000,0.0,0.0,6.962211,1.492272,0.000000,6.867857,9.931770,...,0.0,0.000000,16.229820,0.0,0.000000,2.684118,14.063266,8.427286,0.000000,0.000000
4,RELINE,f4580de8ed4b87729112d8e7779926905ae7b859dbe4ec...,4.586922,0.0,0.0,0.000000,0.000000,6.750409,0.000000,11.644707,...,0.0,13.045745,0.000000,0.0,0.000000,0.806817,0.000000,2.727366,6.296984,4.032376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,FARFLI,de965e33dff58cf011106feacef2f804d9e35d00b8b5ff...,3.101174,0.0,0.0,3.244083,0.000000,6.665556,0.000000,13.080181,...,0.0,6.186675,6.110146,0.0,0.000000,0.000000,0.000000,0.058109,0.000000,0.000000
664,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,15.698259,0.0,0.0,0.000000,13.621684,5.317119,0.000000,0.000000,...,0.0,11.855600,0.000000,0.0,0.000000,0.000000,0.000000,3.220627,0.000000,0.000000
665,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,0.000000,0.0,0.0,1.607330,0.000000,0.000000,4.734159,13.946035,...,0.0,2.253247,4.843779,0.0,0.000000,3.935575,7.204690,11.374994,0.000000,0.000000
666,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,6.559162,0.0,0.0,0.000000,0.000000,0.000000,8.169443,9.100674,...,0.0,10.957123,0.000000,0.0,4.151663,6.178845,7.022230,0.000000,0.000000,0.000000


For training the model, we don't need the `hash` of the sample.

In [5]:
df = raw_df.drop(columns=["hash"])
df

,label,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,dim-08,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,10.802482,0.0,0.0,0.000000,0.000000,12.707260,9.048082,8.721670,0.000000,...,0.0,13.428146,0.000000,0.0,0.000000,0.000000,0.801215,10.506460,0.024712,0.138089
1,DARKKOMET,16.677288,0.0,0.0,0.000000,13.798045,0.000000,3.046454,0.000000,0.000000,...,0.0,11.214973,0.000000,0.0,0.000000,8.833573,6.818852,0.000000,0.000000,0.000000
2,HIVE,0.000000,0.0,0.0,6.604103,2.193231,4.128004,2.006744,6.526801,0.000000,...,0.0,2.912180,10.735530,0.0,0.000000,0.774567,5.789243,0.877978,0.000000,0.000000
3,AGENTB,0.000000,0.0,0.0,6.962211,1.492272,0.000000,6.867857,9.931770,5.388600,...,0.0,0.000000,16.229820,0.0,0.000000,2.684118,14.063266,8.427286,0.000000,0.000000
4,RELINE,4.586922,0.0,0.0,0.000000,0.000000,6.750409,0.000000,11.644707,0.000000,...,0.0,13.045745,0.000000,0.0,0.000000,0.806817,0.000000,2.727366,6.296984,4.032376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,FARFLI,3.101174,0.0,0.0,3.244083,0.000000,6.665556,0.000000,13.080181,0.000000,...,0.0,6.186675,6.110146,0.0,0.000000,0.000000,0.000000,0.058109,0.000000,0.000000
664,TRICKBOT,15.698259,0.0,0.0,0.000000,13.621684,5.317119,0.000000,0.000000,0.000000,...,0.0,11.855600,0.000000,0.0,0.000000,0.000000,0.000000,3.220627,0.000000,0.000000
665,NECONYD,0.000000,0.0,0.0,1.607330,0.000000,0.000000,4.734159,13.946035,10.490717,...,0.0,2.253247,4.843779,0.0,0.000000,3.935575,7.204690,11.374994,0.000000,0.000000
666,TRICKBOT,6.559162,0.0,0.0,0.000000,0.000000,0.000000,8.169443,9.100674,0.000000,...,0.0,10.957123,0.000000,0.0,4.151663,6.178845,7.022230,0.000000,0.000000,0.000000


Separate into `X` and `y`.

In [6]:
X = df.drop(columns=["label"])
y = df["label"]

In [7]:
print(X.shape)
print(y.shape)

(668, 32)
(668,)


# Label Encoding and Train-Test Split

We need to encode the labels for the `y`.

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

We want to save the label encoder for use in the full model.

In [9]:
import pickle

with open("../models/classifier/label-encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

80% of the data will be saved for training, while 20% will be left for testing.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(534, 32)
(134, 32)
(534,)
(134,)


# Model Creation

Nothing too special here, we just use several dense layers for classification.

In [12]:
from keras import layers
from keras.models import Sequential

In [13]:
INITIAL_EPOCH = 0

def create_model():
    model = Sequential(name="Classifier")

    model.add(keras.Input((X.shape[1],), name="input"))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(len(label_encoder.classes_), activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

In [14]:
model = create_model()
model.summary()

2024-04-29 13:42:25.899011: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:42:25.899104: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:42:25.899136: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:42:26.655425: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 13:42:26.655495: I external/local_xla/xla/stream_executor

Model: "Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 52)             │         3,380 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,636 (154.83 KB)

 Trainable params: 39,636 (154.83 KB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [15]:
import os

checkpoint_path = "../models/classifier/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 54
# model = keras.models.load_model(checkpoint_path)

Train the model.

In [18]:
NUM_EPOCHS = 100

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/100


I0000 00:00:1714369347.732986  979563 service.cc:145] XLA service 0x7f12e4004ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714369347.733030  979563 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 13:42:27.754261: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 13:42:28.631892: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/14 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - loss: 5.0027 - sparse_categorical_accuracy: 0.0000e+00

I0000 00:00:1714369350.224885  979563 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 4.2890 - sparse_categorical_accuracy: 0.0386  
Epoch 1: val_loss improved from inf to 3.54252, saving model to ../models/classifier/checkpoint.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 161ms/step - loss: 4.2692 - sparse_categorical_accuracy: 0.0407 - val_loss: 3.5425 - val_sparse_categorical_accuracy: 0.1308
Epoch 2/100
 1/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.2914 - sparse_categorical_accuracy: 0.1875
Epoch 2: val_loss improved from 3.54252 to 3.08723, saving model to ../models/classifier/checkpoint.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.2459 - sparse_categorical_accuracy: 0.2478 - val_loss: 3.0872 - val_sparse_categorical_accuracy: 0.3178
Epoch 3/100
 1/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.0367 - sparse_categorical_accuracy: 0.3750
Epoch 3: val_loss improved from 3.08723 to 2.76595, saving model to ../models/classifier/checkpoint.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.7725 - sparse_categoric

Load the best performing model.

In [19]:
model = keras.models.load_model(checkpoint_path)

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing loss:      {test_loss:5.5f}")
print(f"Testing accuracy:  {test_acc:5.5f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 1.9824 - sparse_categorical_accuracy: 0.5629
Testing loss:      2.09618
Testing accuracy:  0.55970


In [21]:
model.save("../models/classifier/classifier.keras")